In [1]:
from IPython.display import Image, display

import os
import autogen
from autogen.coding import LocalCommandLineCodeExecutor
from autogen_ext.models.openai import OpenAIChatCompletionClient

from dotenv import load_dotenv

load_dotenv()

config_list = [
    {
        'model': 'gpt-4o',
        'api_key':  os.getenv("OPENAI_API_KEY"),
        'tags': ['tool', 'gpt-4'],
    },
    {
        'model': 'gemini-1.5-pro',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['tool', 'gemini'],
    },
    {
        'model': 'gemini-1.5-flash',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['tool', 'gemini'],
    },
    {
        'model': 'gemini-1.0-pro',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['gemini'],
    }
]

import json
config_list_file_name = ".config_list"
with open(config_list_file_name, "w") as file:
    json.dump(config_list, file, indent=4)

gpt_config_list = autogen.config_list_from_json(config_list_file_name, filter_dict={"tags": ["gpt-4"]})
gpt_llm_config = {"config_list": gpt_config_list, "timeout": 120}

gemini_config_list = autogen.config_list_from_json(config_list_file_name, filter_dict={"tags": ["gemini"]})
gemini_llm_config = {"config_list": gemini_config_list, "timeout": 120}

In [2]:
import autogen
from arxiv import Client, SortCriterion, SortOrder
import datetime
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet

# Configuration for arXiv search and PDF generation
PDF_FILENAME = "technical_review.pdf"
MAX_RESULTS = 10  # Number of papers to include
YEARS_BACK = 3    # Look for papers from last N years

def search_arxiv(topic: str):
    """Search arXiv for recent papers on a given topic"""
    current_year = datetime.datetime.now().year
    start_year = current_year - YEARS_BACK
    
    client = Client()
    search = arxiv.Search(
        query=f'"{topic}" AND submittedDate:[{start_year}0101 TO {current_year}1231]',
        max_results=MAX_RESULTS,
        sort_by=SortCriterion.SubmittedDate,
        sort_order=SortOrder.Descending
    )
    
    papers = []
    for result in client.results(search):
        papers.append({
            'title': result.title,
            'authors': ', '.join([a.name for a in result.authors]),
            'summary': result.summary,
            'published': result.published.strftime("%Y-%m-%d"),
            'url': result.pdf_url
        })
    return papers

def generate_pdf(document_content: list):
    """Generate PDF document from formatted content"""
    doc = SimpleDocTemplate(PDF_FILENAME, pagesize=letter)
    styles = getSampleStyleSheet()
    story = []
    
    # Add document title
    story.append(Paragraph("Technical Review Document", styles['Title']))
    story.append(Spacer(1, 24))
    
    # Add publication details
    for idx, paper in enumerate(document_content, 1):
        story.append(Paragraph(f"{idx}. {paper['title']}", styles['Heading2']))
        story.append(Paragraph(f"Authors: {paper['authors']}", styles['Normal']))
        story.append(Paragraph(f"Published: {paper['published']}", styles['Normal']))
        story.append(Paragraph(f"Summary: {paper['summary']}", styles['Normal']))
        story.append(Paragraph(f"URL: {paper['url']}", styles['Normal']))
        story.append(Spacer(1, 12))
    
    doc.build(story)
    return f"PDF document generated at {PDF_FILENAME}"

# Configure AutoGen agents
# config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")
# llm_config = {"config_list": config_list, "timeout": 120}

# Create agent instances
research_assistant = autogen.AssistantAgent(
    name="Researcher",
    system_message="You specialize in searching academic databases. Use search_arxiv for arXiv queries.",
    llm_config=gemini_llm_config,
)

technical_writer = autogen.AssistantAgent(
    name="Technical_Writer",
    system_message="You format technical content into structured documents. Prepare data for PDF generation.",
    llm_config=gemini_llm_config,
)

user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    human_input_mode="NEVER",
    code_execution_config={"work_dir": "."},
    default_auto_reply="Continue",
)

# Register functions with the agents
user_proxy.register_function(
    function_map={
        "search_arxiv": search_arxiv,
        "generate_pdf": generate_pdf,
    }
)

# Initiate the document creation process
user_proxy.initiate_chat(
    research_assistant,
    message=f"""Create a technical review document on quantum computing. 
    Include {MAX_RESULTS} recent papers from the last {YEARS_BACK} years.
    Final output should be a PDF file named {PDF_FILENAME}."""
)

# Continue the process with the technical writer
user_proxy.initiate_chat(
    technical_writer,
    message="Format the research data into a structured document suitable for PDF generation."
)

User_Proxy (to Researcher):

Create a technical review document on quantum computing. 
    Include 10 recent papers from the last 3 years.
    Final output should be a PDF file named technical_review.pdf.

--------------------------------------------------------------------------------
Researcher (to User_Proxy):

I can't create a PDF file directly. I am a text-based AI and don't have the capability to interact with the file system to generate and save PDF documents.  However, I can provide you with the text content for a technical review document on quantum computing, incorporating 10 recent papers (from the last 3 years, at the time of my last training update).  You can then copy this text and paste it into a document editor (like Microsoft Word, Google Docs, or LibreOffice Writer) and export it as a PDF.

**Technical Review: Recent Advances in Quantum Computing**

**Abstract:** This review summarizes recent advancements in quantum computing, focusing on ten key papers published with

KeyboardInterrupt: 